In [ ]:
import os
import numpy as np
import gudhi as gd
import scipy as sp
import pandas as pd
import biopandas as bd
from biopandas.pdb import PandasPdb
import json

file_dir='/data/sd2/protein-design/rocklin_pdbs/'
save_file_dir='/data/sd2/protein-design/rocklin_pers_json'
all_files=os.listdir(file_dir)

for j in range(len(all_files)):
    file_name = all_files[j]
    full_name = file_dir+file_name
    ppdb = PandasPdb()
    ppdb.read_pdb(full_name)

    x_c=ppdb.df['ATOM']['x_coord']
    y_c=ppdb.df['ATOM']['y_coord']
    z_c=ppdb.df['ATOM']['z_coord']
    res_num=ppdb.df['ATOM']['residue_number']
    r_max=max(res_num.values)

    data = {'x_coord': x_c, 'y_coord': y_c, 'z_coord': z_c}
    df = pd.DataFrame(data=data)

    ac = gd.AlphaComplex(points=df.values)
    st = ac.create_simplex_tree()
    st.persistence()

    H_2=st.persistence_intervals_in_dimension(2)
    H_1=st.persistence_intervals_in_dimension(1)
    H_0=st.persistence_intervals_in_dimension(0)

    dicts={}
    dicts[0]= {"Del_res_name":"No_del",   
            "BD2":0,"BD1":0,"BD0":0,
            "Persistence":st.persistence()}

    for i in range(r_max):
        x_c=ppdb.df['ATOM']['x_coord'][ppdb.df['ATOM']['residue_number']!=i+1]
        y_c=ppdb.df['ATOM']['y_coord'][ppdb.df['ATOM']['residue_number']!=i+1]
        z_c=ppdb.df['ATOM']['z_coord'][ppdb.df['ATOM']['residue_number']!=i+1]
    
        del_res=ppdb.df['ATOM']['residue_name'][ppdb.df['ATOM']['residue_number']==i+1]
        del_res_name=del_res.values[0]
        data = {'x_coord': x_c, 'y_coord': y_c, 'z_coord': z_c}
        dfdel = pd.DataFrame(data=data)
    
        acdel = gd.AlphaComplex(points=dfdel.values)
        stdel = acdel.create_simplex_tree()
        stdel.persistence()

        H_2_del=stdel.persistence_intervals_in_dimension(2)
        H_1_del=stdel.persistence_intervals_in_dimension(1)
        H_0_del=stdel.persistence_intervals_in_dimension(0)

        BD2=gd.bottleneck_distance(H_2, H_2_del)
        BD1=gd.bottleneck_distance(H_1, H_1_del)
        BD0=gd.bottleneck_distance(H_0, H_0_del)
    
        dicts[i+1]={"Del_res_name":del_res_name,
                "BD2":BD2,"BD1":BD1,"BD0":BD0,
                "Persistence":stdel.persistence()}
        
    j_data = json.dumps(dicts)

    final_name=save_file_dir+"/"+file_name[:-4]+'_del_pers.json'

    with open(final_name, "w") as outfile: 
        outfile.write(j_data) 